In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
raw_train = pd.read_csv('./data/sales_train.csv.gz')
raw_test = pd.read_csv('./data/test.csv.gz')
items = pd.read_csv('./data/items.csv')
item_cats = pd.read_csv('./data/item_cats.csv')
item_categories = pd.read_csv('./data/item_categories.csv')
shops = pd.read_csv('./data/shops.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv.gz')
raw_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [30]:
print(raw_train.count())
raw_train.head()

date              2935849
date_block_num    2935849
shop_id           2935849
item_id           2935849
item_price        2935849
item_cnt_day      2935849
pd_date           2935849
year              2935849
month             2935849
day               2935849
dtype: int64


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,pd_date,year,month,day
0,02.01.2013,0,59,22154,999.00,1.0,2013-01-02,2013,1,2
1,03.01.2013,0,25,2552,899.00,1.0,2013-01-03,2013,1,3
2,05.01.2013,0,25,2552,899.00,-1.0,2013-01-05,2013,1,5
3,06.01.2013,0,25,2554,1709.05,1.0,2013-01-06,2013,1,6
4,15.01.2013,0,25,2555,1099.00,1.0,2013-01-15,2013,1,15


In [24]:
raw_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [35]:
raw_train['pd_date'] = pd.to_datetime(raw_train['date'], format='%d.%m.%Y')
raw_train['year'] = pd.DatetimeIndex(raw_train['pd_date']).year
raw_train['month'] = pd.DatetimeIndex(raw_train['pd_date']).month
raw_train['day'] = pd.DatetimeIndex(raw_train['pd_date']).day
raw_train['sales'] = raw_train['item_price'] * raw_train['item_cnt_day']
all_train = raw_train.join(
    items, on='item_id', rsuffix='.right'
).drop(columns=['item_name', 'item_id.right', 'date', 'date_block_num', 'pd_date'])
print(all_train.count())
all_train.head()

shop_id             2935849
item_id             2935849
item_price          2935849
item_cnt_day        2935849
year                2935849
month               2935849
day                 2935849
sales               2935849
item_category_id    2935849
dtype: int64


,shop_id,item_id,item_price,item_cnt_day,year,month,day,sales,item_category_id
0,59,22154,999.00,1.0,2013,1,2,999.00,37
1,25,2552,899.00,1.0,2013,1,3,899.00,58
2,25,2552,899.00,-1.0,2013,1,5,-899.00,58
3,25,2554,1709.05,1.0,2013,1,6,1709.05,58
4,25,2555,1099.00,1.0,2013,1,15,1099.00,56


In [39]:
all_train.groupby(
    ['shop_id', 'item_id', 'item_category_id', 'year', 'month']
).agg({'sales': 'sum'}).reset_index().head()

,shop_id,item_id,item_category_id,year,month,sales
0,0,30,40,2013,2,8215.0
1,0,31,37,2013,2,4774.0
2,0,32,40,2013,1,1326.0
3,0,32,40,2013,2,2210.0
4,0,33,37,2013,1,1041.0
